In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('homework4').getOrCreate()

## Данные: [google drive: raw_sales.csv]
(https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
*  Стоимость последнего проданного дома до текущего


In [383]:
data=spark.read.csv('raw_sales.csv',header=True, inferSchema=True)

In [384]:
data.show(5)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-02-07 00:00:00|    2607|525000|       house|       4|
|2007-02-27 00:00:00|    2906|290000|       house|       3|
|2007-03-07 00:00:00|    2905|328000|       house|       3|
|2007-03-09 00:00:00|    2905|380000|       house|       4|
|2007-03-21 00:00:00|    2906|310000|       house|       3|
+-------------------+--------+------+------------+--------+
only showing top 5 rows



In [385]:
data.printSchema()

root
 |-- datesold: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- bedrooms: integer (nullable = true)



In [386]:
from pyspark.sql.functions import avg

In [387]:
#Средняя стомость проданных домов до текущего в том же районе (4digit postcode)
# sales_avg=data.groupBy('postcode').agg(avg('price'))

In [388]:
# sales_avg=data.groupBy('postcode').avg('price')

In [389]:
# посмотрим данные propertyType
data.describe("propertyType").show()

+-------+------------+
|summary|propertyType|
+-------+------------+
|  count|       29580|
|   mean|        null|
| stddev|        null|
|    min|       house|
|    max|        unit|
+-------+------------+



In [390]:
# spark может в sql(но мы тут не используем)
# data.registerTempTable('df')

# spark.sql('''
# select *, 
# row_number() over( partition by postcode order by price ) as rn from df
# ''').show()

In [391]:
    #оконные функци

from pyspark.sql import Window

from pyspark.sql import functions as F

# windSpec=Window().partitionBy('postcode').orderBy('price')

* пронумеруем строки с домами:
1) отфильтруем дома - data['propertyType']=='house'
2) добавим номер поля, - partitionBy('propertyType').orderBy('datesold')

In [392]:
windSpec=Window().partitionBy('propertyType').orderBy('datesold')

In [393]:
data_rn=data.filter(data['propertyType']=='house').withColumn('row_number', F.row_number().over(windSpec))

In [394]:
# data_rn=data_rn.filter(data_rn['propertyType']=='house')

In [395]:
data_rn.describe("propertyType").show()

+-------+------------+
|summary|propertyType|
+-------+------------+
|  count|       24552|
|   mean|        null|
| stddev|        null|
|    min|       house|
|    max|       house|
+-------+------------+



In [396]:
data_rn.show()

+-------------------+--------+-------+------------+--------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|row_number|
+-------------------+--------+-------+------------+--------+----------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|         1|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|         2|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|         3|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|         4|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|         5|
|2007-04-04 00:00:00|    2905| 465000|       house|       4|         6|
|2007-04-24 00:00:00|    2607| 399000|       house|       3|         7|
|2007-04-30 00:00:00|    2606|1530000|       house|       4|         8|
|2007-05-24 00:00:00|    2902| 359000|       house|       3|         9|
|2007-05-25 00:00:00|    2906| 320000|       house|       3|        10|
|2007-06-26 00:00:00|    2902| 385000|       house|       3|    

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
*  Стоимость последнего проданного дома до текущего

In [418]:
windSpec=Window()\
.partitionBy('postcode')\
.orderBy('datesold')\
.rowsBetween(Window.currentRow-10, Window.currentRow)
# .rowsBetween(Window.currentRow, Window.currentRow+10)
#.rangeBetween(Window.currentRow, Window.currentRow)
#.rangeBetween(Window.currentRow-10, Window.currentRow)

In [421]:
data_rn=data_rn.withColumn('10_avg_before', F.avg('Price').over(windSpec))

In [428]:
data_rn.show(5)

+-------------------+--------+-------+------------+--------+----------+-----------------+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|row_number|     10_avg_after|    10_avg_before|
+-------------------+--------+-------+------------+--------+----------+-----------------+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|        20|735590.9090909091|         327000.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|        40|792227.2727272727|         558500.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|       115|786772.7272727273|647333.3333333334|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|       190|         789500.0|         558625.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|       230|859045.4545454546|         599900.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|       270|894045.4545454546|654416.6666666666|
|2008-09-02 00:00:00|    260

In [414]:
windSpec=Window()\
.partitionBy('postcode')\
.orderBy('datesold')\
.rowsBetween(Window.currentRow, Window.currentRow+10)

In [416]:
data_rn=data_rn.withColumn('10_avg_after', F.avg('Price').over(windSpec))

*  Стоимость последнего проданного дома до текущего

In [429]:
windSpec=Window()\
.partitionBy('postcode')\
.orderBy('datesold')\
# .rowsBetween(Window.currentRow, Window.currentRow)

In [430]:
data_rn=data_rn.withColumn('last_sold', F.last('Price').over(windSpec))

In [567]:
data_rn.show(5)

+-------------------+--------+------+------------+--------+----------+-----------------+-----------------+---------+
|           datesold|postcode| price|propertyType|bedrooms|row_number|     10_avg_after|    10_avg_before|last_sold|
+-------------------+--------+------+------------+--------+----------+-----------------+-----------------+---------+
|2007-07-08 00:00:00|    2600|327000|       house|       1|        20|735590.9090909091|         327000.0|   327000|
|2007-08-16 00:00:00|    2600|790000|       house|       4|        40|792227.2727272727|         558500.0|   790000|
|2007-12-05 00:00:00|    2600|825000|       house|       3|       115|786772.7272727273|647333.3333333334|   825000|
|2008-04-24 00:00:00|    2600|292500|       house|       1|       190|         789500.0|         558625.0|   292500|
|2008-06-19 00:00:00|    2600|765000|       house|       5|       230|859045.4545454546|         599900.0|   765000|
+-------------------+--------+------+------------+--------+-----

### Задание 2
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*   Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*   Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*   Стоимость последнего проданного дома до текущего ((1 балл)
*   и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [542]:
from pyspark.sql.types import IntegerType,StringType,ArrayType,FloatType

In [680]:
#testing....sum of rows
index=['datesold', 'postcode', 'price', 'propertyType', 'bedrooms', 'row_number',
       '10_avg_after', '10_avg_before', 'last_sold']

def f(row):
    return len(set(row))


udf_f = F.udf(f, returnType=(StringType()))

data_rn.withColumn('value', udf_f(F.array(data_rn['datesold'],data_rn['postcode'],data_rn['price'],data_rn['propertyType'],data_rn['bedrooms'],data_rn['row_number'],data_rn['10_avg_after'],data_rn['10_avg_before'],data_rn['last_sold']))).show()

+-------------------+--------+-------+------------+--------+----------+-----------------+-----------------+---------+-----+
|           datesold|postcode|  price|propertyType|bedrooms|row_number|     10_avg_after|    10_avg_before|last_sold|value|
+-------------------+--------+-------+------------+--------+----------+-----------------+-----------------+---------+-----+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|        20|735590.9090909091|         327000.0|   327000|    8|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|        40|792227.2727272727|         558500.0|   790000|    8|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|       115|786772.7272727273|647333.3333333334|   825000|    8|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|       190|         789500.0|         558625.0|   292500|    8|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|       230|859045.4545454546|         599900.0|   765000|    8|
|2008-07

# Задание 3
SQL like case when или if elif else

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

Если одно из полей Null, запишите в эту колонку "Нет данных"

In [655]:
data_rn.select(data_rn['10_avg_after'],data_rn['10_avg_before'],F.when(data_rn['10_avg_after'] > data_rn['10_avg_before'],'+')\
.when(data_rn['10_avg_after'] < data_rn['10_avg_before'],'-')\
.when(data_rn['10_avg_after'] == data_rn['10_avg_before'],'=')\
.otherwise('Нет данных')\
.alias('col'))\
.show()

+-----------------+-----------------+---+
|     10_avg_after|    10_avg_before|col|
+-----------------+-----------------+---+
|735590.9090909091|         327000.0|  +|
|792227.2727272727|         558500.0|  +|
|786772.7272727273|647333.3333333334|  +|
|         789500.0|         558625.0|  +|
|859045.4545454546|         599900.0|  +|
|894045.4545454546|654416.6666666666|  +|
|862045.4545454546|758071.4285714285|  +|
|816590.9090909091|         755812.5|  +|
|841590.9090909091|751833.3333333334|  +|
|841772.7272727273|         745650.0|  +|
|         834500.0|735590.9090909091|  +|
|838136.3636363636|792227.2727272727|  +|
|842681.8181818182|786772.7272727273|  +|
|845409.0909090909|         789500.0|  +|
|868590.9090909091|859045.4545454546|  +|
|846090.9090909091|894045.4545454546|  -|
|823363.6363636364|862045.4545454546|  -|
|844909.0909090909|816590.9090909091|  +|
|852181.8181818182|841590.9090909091|  +|
|826727.2727272727|841772.7272727273|  -|
+-----------------+---------------